# Chuyển oco2 nc4 thành csv


### 1. Import thư viện


In [5]:
import xarray as xr # Thư viện xử lý dữ liệu dạng lưới (grid)(.nc, .nc4)
import pandas as pd # Thư viện xử lý dữ liệu dạng bảng (.csv)
import glob # Thư viện tìm kiếm file theo mẫu 
import os # Thư viện xử lý file và thư mục
import geopandas as gpd # Thư viện xử lý dữ liệu không gian
from datetime import datetime, timedelta # Thư viện xử lý thời gian
from shapely.geometry import Point # thư viện xử lý dữ liệu hình học


### 2. Đọc và kiểm tra cấu trúc dữ liệu CO₂ trong file .nc4


In [ ]:
# Xác định file .nc4 và đọc dữ liệu bằng xarray
# Đường dẫn tới file .nc4
file_path = r"E:\DownloadData\co2_nasa\data\nc4\oco2_nc4_2021\oco2_LtCO2_201231_B11210Ar_240913234539s.nc4"  

# Mở file .nc4
ds = xr.open_dataset(file_path)

# In thông tin toàn bộ dataset
print(ds)

# In danh sách các biến
print("\n🔹 Danh sách biến:")
print(list(ds.data_vars))

# In các chiều dữ liệu
print("\n🔹 Các chiều:")
print(list(ds.dims))

# In thuộc tính toàn cục
print("\n🔹 Global attributes:")
print(ds.attrs)

# Đóng file sau khi đọc
ds.close()


### 3. Chuẩn bị đọc nhiều dữ liệu nc4 cùng lúc

In [ ]:
# Đường dẫn đến thư mục chứa các file .nc4
folder_path = "E:\\DownloadData\\co2_nasa\\data\\nc4\\oco3_nc4_2024\\*.nc4"

# Lấy danh sách tất cả các file .nc4
file_list = glob.glob(folder_path) 

# Danh sách lưu DataFrame từ mỗi file
df_list = []

### 4. Đọc, trích xuất và xuất dữ liệu CO₂ (OCO-3. OCO-2)

In [ ]:
for file in file_list: # Lặp qua từng file
    filename = os.path.basename(file) # Lấy tên file từ đường dẫn
    date_str = filename.split('_')[2] 
    full_date = f"20{date_str[:2]}-{date_str[2:4]}-{date_str[4:6]}" 
    # Thử đọc file và trích xuất dữ liệu
    try: 
        ds = xr.open_dataset(file)
        df = pd.DataFrame({
            "file_date": full_date,
            "latitude": ds["latitude"].values,
            "longitude": ds["longitude"].values,
            "sza": ds["solar_zenith_angle"].values,
            "vza": ds["sensor_zenith_angle"].values,
            "xco2": ds["xco2"].values,
            "xco2_sigma": ds["xco2_uncertainty"].values,
            "xco2_qual": ds["xco2_quality_flag"].values,
            "time": ds["time"].values,
        })
        df_list.append(df)
    # Bắt lỗi nếu có
    except Exception as e: 
        print(f"❌ Lỗi với file {file}: {e}")

# Kết hợp tất cả DataFrame thành một DataFrame duy nhất
df_final = pd.concat(df_list, ignore_index=True)

# Xuất CSV
output_csv = r"E:\DownloadData\co2_nasa\data\csv\2024\oco3\oco3_all_2024.csv"
df_final.to_csv(output_csv, index=False)

print(f"✅ Xuất dữ liệu thành công ra {output_csv}!")
print(f"📂 Tổng số file đã xử lý: {len(file_list)}")
print(f"📊 Tổng số dòng dữ liệu: {len(df_final)}")


❌ Lỗi với file E:\DownloadData\co2_nasa\data\nc4\oco3_nc4_2024\oco3_LtCO2_241227_B11072Ar_250206192417s.nc4: [Errno -101] NetCDF: HDF error: b'E:\\DownloadData\\co2_nasa\\data\\nc4\\oco3_nc4_2024\\oco3_LtCO2_241227_B11072Ar_250206192417s.nc4'
✅ Xuất dữ liệu thành công ra E:\DownloadData\co2_nasa\data\csv\2024\oco3\oco3_all_2024.csv!
📂 Tổng số file đã xử lý: 165
📊 Tổng số dòng dữ liệu: 21758432


### 5. Đọc 1 file .nc4 lỗi khi tải ở mục 4(nếu không lỗi không phải chạy code này) và xuất ra CSV

In [ ]:
# Đường dẫn tới file .nc4 cụ thể
file_path = r"E:\oco3_LtCO2_241227_B11072Ar_250206192417s.nc4"

# Lấy ngày từ tên file
file_name = os.path.basename(file_path)
full_date = file_name.split("_")[2]
full_date = f"20{date_str[:2]}-{date_str[2:4]}-{date_str[4:6]}"  # → '2024-12-27'

# Mở file .nc4 trích xuất dữ liệu
ds = xr.open_dataset(file_path)

# Tạo DataFrame từ các biến cần thiết
df = pd.DataFrame({
    "file_date": full_date,
    "latitude": ds["latitude"].values,
    "longitude": ds["longitude"].values,
    "sza": ds["solar_zenith_angle"].values,
    "vza": ds["sensor_zenith_angle"].values,
    "xco2": ds["xco2"].values,
    "xco2_sigma": ds["xco2_uncertainty"].values,
    "xco2_qual": ds["xco2_quality_flag"].values,
    "time": ds["time"].values
})

# Xuất DataFrame ra file CSV
output_csv = r"E:\DownloadData\co2_nasa\data\csv\oco3_241227.csv"
df.to_csv(output_csv, index=False, encoding="utf-8") 

print("✅ Đã trích xuất thành công!")
print(f"📂 File CSV: {output_csv}")
print(f"📊 Số dòng dữ liệu: {len(df)}")


✅ Đã trích xuất thành công!
📂 File CSV: E:\DownloadData\co2_nasa\data\csv\oco3_241227.csv
📊 Số dòng dữ liệu: 239372


### 6. Xuất dữ liệu CO2 từ OCO2_all lọc theo ranh giới Việt Nam sang file OCO2_vietnam


In [12]:
# Đường dẫn các file 
input_csv_path = r"E:\DownloadData\co2_nasa\data\csv\2024\oco3\oco3_all_2024.csv"
shapefile_path = r"E:\RanhGioi\VNM_adm\gadm41_VNM_0.shp"
output_csv_path = r"E:\DownloadData\co2_nasa\data\csv\2024\oco3\oco3_vietnam_2024.csv"

# Đọc shapefile ranh giới Việt Nam
gdf_vn = gpd.read_file(shapefile_path)
if gdf_vn.crs != "EPSG:4326":
    gdf_vn = gdf_vn.to_crs(epsg=4326)

# Lấy bounding box để lọc sơ bộ
min_lon, min_lat, max_lon, max_lat = gdf_vn.total_bounds

# Nếu file đã tồn tại thì xóa để ghi mới
if os.path.exists(output_csv_path):
    os.remove(output_csv_path)

# Đọc và xử lý từng phần nhỏ
chunk_size = 500_000  # có thể thay đổi tùy theo RAM
chunk_idx = 0

for chunk in pd.read_csv(input_csv_path, chunksize=chunk_size):
    chunk_idx += 1
    print(f"🔄 Đang xử lý phần {chunk_idx}...")

    # Lọc sơ bộ theo bounding box
    chunk = chunk[
        (chunk["longitude"] >= min_lon) & (chunk["longitude"] <= max_lon) &
        (chunk["latitude"] >= min_lat) & (chunk["latitude"] <= max_lat)
    ]

    # Tạo GeoDataFrame từ chunk đã lọc
    geometry = [Point(xy) for xy in zip(chunk["longitude"], chunk["latitude"])]
    gdf_chunk = gpd.GeoDataFrame(chunk, geometry=geometry, crs="EPSG:4326")

    # Lọc chính xác bằng spatial join
    gdf_filtered = gpd.sjoin(gdf_chunk, gdf_vn, predicate="within", how="inner")

    # Xóa các cột không cần
    gdf_filtered.drop(columns=["geometry", "index_right"], errors="ignore", inplace=True)

    # Ghi kết quả vào file (append)
    gdf_filtered.to_csv(output_csv_path, mode='a', index=False, header=not os.path.exists(output_csv_path))

    print(f"✅ Đã xử lý xong phần {chunk_idx}, ghi {len(gdf_filtered)} dòng.")

print("🎉 Hoàn tất lọc toàn bộ dữ liệu!")

🔄 Đang xử lý phần 1...
✅ Đã xử lý xong phần 1, ghi 4 dòng.
🔄 Đang xử lý phần 2...
✅ Đã xử lý xong phần 2, ghi 10 dòng.
🔄 Đang xử lý phần 3...
✅ Đã xử lý xong phần 3, ghi 0 dòng.
🔄 Đang xử lý phần 4...
✅ Đã xử lý xong phần 4, ghi 0 dòng.
🔄 Đang xử lý phần 5...
✅ Đã xử lý xong phần 5, ghi 113 dòng.
🔄 Đang xử lý phần 6...
✅ Đã xử lý xong phần 6, ghi 0 dòng.
🔄 Đang xử lý phần 7...
✅ Đã xử lý xong phần 7, ghi 203 dòng.
🔄 Đang xử lý phần 8...
✅ Đã xử lý xong phần 8, ghi 17 dòng.
🔄 Đang xử lý phần 9...
✅ Đã xử lý xong phần 9, ghi 13 dòng.
🔄 Đang xử lý phần 10...
✅ Đã xử lý xong phần 10, ghi 0 dòng.
🔄 Đang xử lý phần 11...
✅ Đã xử lý xong phần 11, ghi 0 dòng.
🔄 Đang xử lý phần 12...
✅ Đã xử lý xong phần 12, ghi 0 dòng.
🔄 Đang xử lý phần 13...
✅ Đã xử lý xong phần 13, ghi 403 dòng.
🔄 Đang xử lý phần 14...
✅ Đã xử lý xong phần 14, ghi 28 dòng.
🔄 Đang xử lý phần 15...
✅ Đã xử lý xong phần 15, ghi 554 dòng.
🔄 Đang xử lý phần 16...
✅ Đã xử lý xong phần 16, ghi 0 dòng.
🔄 Đang xử lý phần 17...
✅ Đã x

###  7. Lọc dữ liệu theo chấy lượng(quality flag) xco2_qual 

In [10]:
# Đọc file CSV đã lọc

df = pd.read_csv(r"E:\DownloadData\co2_nasa\data\csv\2023\oco2\oco2_vietnam_2023.csv")
# Kiểm tra cột có tên đúng không
print(df.columns)

# Kiểm tra giá trị duy nhất trong cột xco2_qual
print(df['xco2_qual'].unique())


Index(['file_date', 'latitude', 'longitude', 'sza', 'vza', 'xco2',
       'xco2_sigma', 'xco2_qual', 'time', 'GID_0', 'COUNTRY'],
      dtype='object')
[1. 0.]


In [17]:
# Đọc file
df = pd.read_csv(r"E:\DownloadData\co2_nasa\data\csv\2024\oco3\oco3_vietnam_2024.csv")

# Lọc các dòng có xco2_qual = 0
filtered_df = df[df['xco2_qual'] == 0.0]  # dùng 0.0 hoặc 0 đều được

# Kiểm tra số dòng lọc được
print(f"Số dòng sau khi lọc xco2_qual = 0: {len(filtered_df)}")

# Xuất ra CSV
filtered_df.to_csv(r"E:\DownloadData\co2_nasa\data\csv\2024\oco3\oco3_vn_qual0_2024.csv", index=False)


Số dòng sau khi lọc xco2_qual = 0: 1931


### 7.Gộp dữ liệu file oco3 và oco2


In [19]:

# Đọc 2 file CSV
df1 = pd.read_csv(r"E:\DownloadData\co2_nasa\data\csv\2024\oco2\oco2_vn_qual0_2024.csv")
df2 = pd.read_csv(r"E:\DownloadData\co2_nasa\data\csv\2024\oco3\oco3_vn_qual0_2024.csv")

# Gộp 2 file
df_merged = pd.concat([df1, df2], ignore_index=True)

# Chuyển cột 'file_date' về dạng datetime (tự động nhận dạng định dạng ngày)
df_merged['file_date'] = pd.to_datetime(df_merged['file_date'], errors='coerce')

# Kiểm tra nếu có dòng lỗi định dạng ngày
invalid_dates = df_merged[df_merged['file_date'].isna()]
if not invalid_dates.empty:
    print("⚠️ Có dòng lỗi định dạng ngày, sẽ bị xoá:")
    print(invalid_dates)

# Xoá các dòng có giá trị NaT ở 'file_date'
df_merged = df_merged.dropna(subset=['file_date'])

# Sắp xếp theo ngày tăng dần
df_merged = df_merged.sort_values(by='file_date').reset_index(drop=True)

# Xuất file CSV đã gộp và sắp xếp
df_merged.to_csv(r"E:\DownloadData\co2_nasa\data\csv\2024\oco_merged_2024.csv", index=False)

print("✅ Đã gộp và sắp xếp xong dữ liệu!")

✅ Đã gộp và sắp xếp xong dữ liệu!


In [21]:


# Đọc file đầy đủ (nhưng thiếu 2 ngày)
df_full = pd.read_csv(r"E:\DownloadData\co2_nasa\data\csv\2024\oco_merged_2024.csv")

# Đọc hai file bổ sung
df_0525 = pd.read_csv(r"E:\DownloadData\co2_nasa\data\csv\oco2_240525_vn_qual0.csv")
df_0705 = pd.read_csv(r"E:\DownloadData\co2_nasa\data\csv\oco2_240705_vn_qual0.csv")

# Gộp 2 file bổ sung lại
df_missing = pd.concat([df_0525, df_0705], ignore_index=True)

# Gộp toàn bộ vào file chính
df_all = pd.concat([df_full, df_missing], ignore_index=True)

# Sắp xếp theo ngày (file_date)
df_all = df_all.sort_values(by="file_date")

# Ghi ra file mới
df_all.to_csv(r"E:\DownloadData\co2_nasa\data\csv\2024\oco_merged_2024_final.csv", index=False)
